In [1]:
import json
import jsonlines
import statistics
from tqdm import tqdm
import openai
openai.api_key = ''

In [2]:
with open('inference_data.json','r') as f: #inference_data.json
    wiki = json.load(f)

In [3]:
wiki[0]

{'title': 'Czechs in Argentina',
 'answer': ['Czechs in Argentina The Czech immigration in Argentina began during the World War 1 and it was split in four periods.',
  'It is estimated that around 40,000 Czechs arrived to Argentina until 1970.',
  'Argentina has the largest Czech community in Latin America.',
  'Czechs settled mainly in Buenos Aires, Gran La Plata, Rosario and Chaco.',
  'There are four Czech immigration periods in Argentina.',
  'The first one is slightly before to the World War 1, the second one from 1920 to 1930, the third one during the World War 2 and the fourth one, the smallest in migration proportion, during 1990. During the first two periods, the immigration group was mainly integrated by workers and farmers motivated by economic reasons.']}

In [4]:
print(len(wiki))

113678


In [5]:
num = 100

# base prompt 사용

In [6]:

WikiDialog_LLM = []

for i in tqdm(range(num)):
    current_dialog = []
    tmp = ''
    for j in range(len(wiki[i]['answer'])):
        answer = wiki[i]['answer'][j]

        base = f"\
        Please fill in only [BLANK] in the next dialogue.\n \
        [START]\n "

        QA_pair = f"\
        A: [BLANK]\n \
        B: {answer}\n \
        [END]"
        
        
        PROMPT = base + tmp + QA_pair
#         print(PROMPT)
        messages = [{"role": "system", "content": PROMPT}]
        response = openai.chat.completions.create(
                model="gpt-4",
                messages=messages,
                temperature=0,
                max_tokens=150)
        question = response.choices[0].message.content
        question = question.replace("A: ","").replace('\\', '').replace('"','').strip() # remove A and else...
        
        tmp = tmp +  f"\nA: {question}\n B: {answer}"
        current_dialog.append({"question": question, "answer": answer})
    WikiDialog_LLM.append(current_dialog)

100%|██████████| 100/100 [40:26<00:00, 24.27s/it]  


In [7]:
print(base)

        Please fill in only [BLANK] in the next dialogue.
         [START]
 


In [13]:
WikiDialog_LLM

[[{'question': 'Can you tell me about the history of Czech immigration in Argentina?',
   'answer': 'Czechs in Argentina The Czech immigration in Argentina began during the World War 1 and it was split in four periods.'},
  {'question': 'How many Czechs are estimated to have immigrated to Argentina?',
   'answer': 'It is estimated that around 40,000 Czechs arrived to Argentina until 1970.'},
  {'question': 'What is the status of the Czech community in Argentina today?',
   'answer': 'Argentina has the largest Czech community in Latin America.'},
  {'question': 'What areas did the Czechs primarily settle in?',
   'answer': 'Czechs settled mainly in Buenos Aires, Gran La Plata, Rosario and Chaco.'},
  {'question': 'What are the four periods of Czech immigration in Argentina?',
   'answer': 'There are four Czech immigration periods in Argentina.'},
  {'question': 'Can you tell me more about the reasons for immigration during the third and fourth periods?',
   'answer': 'The first one is s

# with instruction

In [14]:
instruction = f"You are an automatic assistant that generates appropriate question based on the predefined answer.\n"

In [15]:

WikiDialog_LLM_instruction = []

for i in tqdm(range(num)):
    current_dialog = []
    tmp = ''
    for j in range(len(wiki[i]['answer'])):
        answer = wiki[i]['answer'][j]
        
        
        base = f"\
        Please fill in only [BLANK] in the next dialogue.\n \
        [START]\n "

        QA_pair = f"\
        A: [BLANK]\n \
        B: {answer}\n \
        [END]"
        
        
        PROMPT = instruction + base + tmp + QA_pair
        #print(PROMPT)
        messages = [{"role": "system", "content": PROMPT}]
        response = openai.chat.completions.create(
                model="gpt-4",
                messages=messages,
                temperature=0,
                max_tokens=150)
        question = response.choices[0].message.content
        question = question.replace("A: ","").replace('\\', '').replace('"','').strip() # remove A and else...
        
        
        tmp = tmp +  f"A: {question}\n B: {answer}\n"
        current_dialog.append({"question": question, "answer": answer})
    WikiDialog_LLM_instruction.append(current_dialog)

100%|██████████| 100/100 [22:47<00:00, 13.68s/it]  


In [16]:
WikiDialog_LLM_instruction

[[{'question': 'Can you tell me about the history of Czech immigration in Argentina?',
   'answer': 'Czechs in Argentina The Czech immigration in Argentina began during the World War 1 and it was split in four periods.'},
  {'question': 'How many Czechs are estimated to have immigrated to Argentina by 1970?',
   'answer': 'It is estimated that around 40,000 Czechs arrived to Argentina until 1970.'},
  {'question': 'Which Latin American country has the largest Czech community?',
   'answer': 'Argentina has the largest Czech community in Latin America.'},
  {'question': 'Where did the majority of Czechs settle in Argentina?',
   'answer': 'Czechs settled mainly in Buenos Aires, Gran La Plata, Rosario and Chaco.'},
  {'question': 'What are the four periods of Czech immigration in Argentina?',
   'answer': 'There are four Czech immigration periods in Argentina.'},
  {'question': 'Can you describe the characteristics of the immigrants during the first two periods of Czech immigration in Arg

# with instruction & passage title

In [17]:

WikiDialog_LLM_instruction_passage_title = []

for i in tqdm(range(num)):
    current_dialog = []
    tmp = ''
    dialog_topic = wiki[i]['title']
    for j in range(len(wiki[i]['answer'])):
        answer = wiki[i]['answer'][j]
        
        base = f"\
        Please fill in only [BLANK] in the next dialogue.\n \
        [START]\n "

        QA_pair = f"\
        A: [BLANK]\n \
        B: {answer}\n \
        [END]"
        title_hint = f"The conversation is about {dialog_topic}.\n"
        
        
        PROMPT = instruction + title_hint + base + tmp + QA_pair
        print(PROMPT)
        messages = [{"role": "system", "content": PROMPT}]
        response = openai.chat.completions.create(
                model="gpt-4",
                messages=messages,
                temperature=0,
                max_tokens=150)
        question = response.choices[0].message.content 
        question = question.replace("A: ","").replace('\\', '').replace('"','').strip() # remove A and else...
        
        
        tmp = tmp +  f"A: {question}\n B: {answer}\n"
        current_dialog.append({"question": question, "answer": answer})
    WikiDialog_LLM_instruction_passage_title.append(current_dialog)

  0%|          | 0/100 [00:00<?, ?it/s]

You are an automatic assistant that generates appropriate question based on the predefined answer.
The conversation is about Czechs in Argentina.
        Please fill in only [BLANK] in the next dialogue.
         [START]
         A: [BLANK]
         B: Czechs in Argentina The Czech immigration in Argentina began during the World War 1 and it was split in four periods.
         [END]
You are an automatic assistant that generates appropriate question based on the predefined answer.
The conversation is about Czechs in Argentina.
        Please fill in only [BLANK] in the next dialogue.
         [START]
 A: Can you tell me about the history of Czech immigration in Argentina?
 B: Czechs in Argentina The Czech immigration in Argentina began during the World War 1 and it was split in four periods.
        A: [BLANK]
         B: It is estimated that around 40,000 Czechs arrived to Argentina until 1970.
         [END]
You are an automatic assistant that generates appropriate question based on t

  1%|          | 1/100 [00:09<15:16,  9.25s/it]

You are an automatic assistant that generates appropriate question based on the predefined answer.
The conversation is about Common ground (communication technique).
        Please fill in only [BLANK] in the next dialogue.
         [START]
         A: [BLANK]
         B: Common ground (communication technique) Common ground is the overlap between the positions of parties that may otherwise disagree.
         [END]
You are an automatic assistant that generates appropriate question based on the predefined answer.
The conversation is about Common ground (communication technique).
        Please fill in only [BLANK] in the next dialogue.
         [START]
 A: Can you explain what is meant by the term 'Common ground' in the context of communication techniques?
 B: Common ground (communication technique) Common ground is the overlap between the positions of parties that may otherwise disagree.
        A: [BLANK]
         B: " Finding common ground" is a technique people use to facilitate int

  2%|▏         | 2/100 [00:21<18:06, 11.09s/it]

...

 98%|█████████▊| 98/100 [34:26<04:54, 147.31s/it]

You are an automatic assistant that generates appropriate question based on the predefined answer.
The conversation is about Gândara.
        Please fill in only [BLANK] in the next dialogue.
         [START]
         A: [BLANK]
         B: Gândara Gândara is a neighbourhood of the Portuguese city of Póvoa de Varzim.
         [END]
You are an automatic assistant that generates appropriate question based on the predefined answer.
The conversation is about Gândara.
        Please fill in only [BLANK] in the next dialogue.
         [START]
 A: Can you tell me what Gândara is?
 B: Gândara Gândara is a neighbourhood of the Portuguese city of Póvoa de Varzim.
        A: [BLANK]
         B: Gândara was between the 16th century and the 18th century, the main reason for disputes between Póvoa de Varzim town hall and Barcelos of the House of Braganza, as it already had a considerable population.
         [END]
You are an automatic assistant that generates appropriate question based on the predef

 99%|█████████▉| 99/100 [34:36<01:46, 106.20s/it]

You are an automatic assistant that generates appropriate question based on the predefined answer.
The conversation is about Arthur F. Hebard.
        Please fill in only [BLANK] in the next dialogue.
         [START]
         A: [BLANK]
         B: Arthur F. Hebard  Arthur Foster Hebard (born 2 March 1940) is Distinguished Professor of Physics at University of Florida in Gainesville, Florida.
         [END]
You are an automatic assistant that generates appropriate question based on the predefined answer.
The conversation is about Arthur F. Hebard.
        Please fill in only [BLANK] in the next dialogue.
         [START]
 A: Who is Arthur F. Hebard?
 B: Arthur F. Hebard  Arthur Foster Hebard (born 2 March 1940) is Distinguished Professor of Physics at University of Florida in Gainesville, Florida.
        A: [BLANK]
         B: He is particularly noted for leading the discovery of superconductivity in Buckminsterfullerene in 1991.
         [END]
You are an automatic assistant that gen

100%|██████████| 100/100 [34:46<00:00, 20.86s/it]


# One shot 

In [32]:
# sample = """
# 'You are an automatic assistant that generates appropriate question based on the predefined answer.
# The conversation is about Arthur F. Hebard.
# Please fill in only [BLANK] in the next dialogue.
# [START] 
# A: Who is Arthur F. Hebard?
# B: Arthur F. Hebard  Arthur Foster Hebard (born 2 March 1940) is Distinguished Professor of Physics at University of Florida in Gainesville, Florida.
# A: What is Arthur F. Hebard particularly noted for?
# B: He is particularly noted for leading the discovery of superconductivity in Buckminsterfullerene in 1991.
# A: Where did Arthur F. Hebard receive his education?
# B: Art Hebard attended The Hotchkiss School and graduated with a BA in Physics from Yale University in 1962.
# A: Who was Arthur F. Hebard\'s PhD advisor and what was his thesis about?
# B: He obtained his PhD from Stanford University in 1971 under William M. Fairbank with thesis "Search for fractional charge using low temperature techniques".
# A: What was Arthur F. Hebard\'s career path after completing his PhD?
# B: After a spell as a Research Associate at Stanford, he became a member of the Technical Staff at AT&T Bell Telephone Laboratories.
# A: [BLANK]
# B: He moved to the University of Florida as a Professor in 1996, and in 2007 was given the title of Distinguished Professor.
# [END]'
# """

In [39]:
shot_1 = """
Please fill in only [BLANK] in the next dialogue.
[START] 
A: Who is Michael Jordan?
B: Michael Jeffrey Jordan also known by his initials MJ, is an American businessman and former professional basketball player. 
A: Is Michael Jordan the greatest player?
B: His profile on the official National Basketball Association (NBA) website states that 'by acclamation, Michael Jordan is the greatest basketball player of all time.'
A: How many games did Michael Jordan play and won?
B: He played fifteen seasons in the NBA, winning six NBA championships with the Chicago Bulls. 
A: [BLANK]
B: He was integral in popularizing the sport of basketball and the NBA around the world in the 1980s and 1990s, becoming a global cultural icon.
[END]
How could Michael Jordan be described in terms of global influence?


"""

In [42]:

WikiDialog_LLM_shot = []

for i in tqdm(range(num)):
    current_dialog = []
    tmp = ''
    for j in range(len(wiki[i]['answer'])):
        answer = wiki[i]['answer'][j]
        
        
        shot = shot_1
        
        
        base = f"\
        Please fill in only [BLANK] in the next dialogue.\n \
        [START]\n "

        QA_pair = f"\
        A: [BLANK]\n \
        B: {answer}\n \
        [END]"
        
        
        PROMPT = shot + base + tmp + QA_pair
        #print(PROMPT)
        messages = [{"role": "system", "content": PROMPT}]
        response = openai.chat.completions.create(
                model="gpt-4",
                messages=messages,
                temperature=0,
                max_tokens=150)
        question = response.choices[0].message.content
        question = question.replace("A: ","").replace('\\', '').replace('"','').strip() # remove A and else...
        
        
        tmp = tmp +  f"A: {question}\n B: {answer}\n"
        current_dialog.append({"question": question, "answer": answer})
    WikiDialog_LLM_shot.append(current_dialog)

100%|██████████| 100/100 [11:53<00:00,  7.14s/it]


In [41]:
WikiDialog_LLM_shot

[[{'question': 'What can you tell me about Czech immigration in Argentina?',
   'answer': 'Czechs in Argentina The Czech immigration in Argentina began during the World War 1 and it was split in four periods.'},
  {'question': 'How many Czechs immigrated to Argentina?',
   'answer': 'It is estimated that around 40,000 Czechs arrived to Argentina until 1970.'},
  {'question': 'What is the significance of the Czech community in Argentina?',
   'answer': 'Argentina has the largest Czech community in Latin America.'},
  {'question': 'Where did the Czechs mainly settle in Argentina?',
   'answer': 'Czechs settled mainly in Buenos Aires, Gran La Plata, Rosario and Chaco.'},
  {'question': 'Can you tell me more about these four periods of Czech immigration?',
   'answer': 'There are four Czech immigration periods in Argentina.'},
  {'question': 'What were the main reasons for Czech immigration during these periods?',
   'answer': 'The first one is slightly before to the World War 1, the secon

# Save json

In [18]:
WikiDialog_LLM[:2]

[[{'question': 'Can you tell me about the history of Czech immigration in Argentina?',
   'answer': 'Czechs in Argentina The Czech immigration in Argentina began during the World War 1 and it was split in four periods.'},
  {'question': 'How many Czechs are estimated to have immigrated to Argentina?',
   'answer': 'It is estimated that around 40,000 Czechs arrived to Argentina until 1970.'},
  {'question': 'What is the status of the Czech community in Argentina today?',
   'answer': 'Argentina has the largest Czech community in Latin America.'},
  {'question': 'What areas did the Czechs primarily settle in?',
   'answer': 'Czechs settled mainly in Buenos Aires, Gran La Plata, Rosario and Chaco.'},
  {'question': 'What are the four periods of Czech immigration in Argentina?',
   'answer': 'There are four Czech immigration periods in Argentina.'},
  {'question': 'Can you tell me more about the reasons for immigration during the third and fourth periods?',
   'answer': 'The first one is s

In [19]:
WikiDialog_LLM_instruction[:2]

[[{'question': 'Can you tell me about the history of Czech immigration in Argentina?',
   'answer': 'Czechs in Argentina The Czech immigration in Argentina began during the World War 1 and it was split in four periods.'},
  {'question': 'How many Czechs are estimated to have immigrated to Argentina by 1970?',
   'answer': 'It is estimated that around 40,000 Czechs arrived to Argentina until 1970.'},
  {'question': 'Which Latin American country has the largest Czech community?',
   'answer': 'Argentina has the largest Czech community in Latin America.'},
  {'question': 'Where did the majority of Czechs settle in Argentina?',
   'answer': 'Czechs settled mainly in Buenos Aires, Gran La Plata, Rosario and Chaco.'},
  {'question': 'What are the four periods of Czech immigration in Argentina?',
   'answer': 'There are four Czech immigration periods in Argentina.'},
  {'question': 'Can you describe the characteristics of the immigrants during the first two periods of Czech immigration in Arg

In [20]:
WikiDialog_LLM_instruction_passage_title[:2]

[[{'question': 'Can you tell me about the history of Czech immigration in Argentina?',
   'answer': 'Czechs in Argentina The Czech immigration in Argentina began during the World War 1 and it was split in four periods.'},
  {'question': 'How many Czechs are estimated to have immigrated to Argentina by 1970?',
   'answer': 'It is estimated that around 40,000 Czechs arrived to Argentina until 1970.'},
  {'question': 'Which Latin American country has the largest Czech community?',
   'answer': 'Argentina has the largest Czech community in Latin America.'},
  {'question': 'Where did the Czechs mainly settle in Argentina?',
   'answer': 'Czechs settled mainly in Buenos Aires, Gran La Plata, Rosario and Chaco.'},
  {'question': 'Can you elaborate on the four periods of Czech immigration in Argentina?',
   'answer': 'There are four Czech immigration periods in Argentina.'},
  {'question': 'What were the characteristics of each of these immigration periods?',
   'answer': 'The first one is sli

In [43]:
WikiDialog_LLM_shot[:2]

[[{'question': 'What can you tell me about Czech immigration in Argentina?',
   'answer': 'Czechs in Argentina The Czech immigration in Argentina began during the World War 1 and it was split in four periods.'},
  {'question': 'How many Czechs immigrated to Argentina?',
   'answer': 'It is estimated that around 40,000 Czechs arrived to Argentina until 1970.'},
  {'question': 'What is the significance of the Czech community in Argentina?',
   'answer': 'Argentina has the largest Czech community in Latin America.'},
  {'question': 'Where did the Czechs mainly settle in Argentina?',
   'answer': 'Czechs settled mainly in Buenos Aires, Gran La Plata, Rosario and Chaco.'},
  {'question': 'What are the four periods of Czech immigration in Argentina?',
   'answer': 'There are four Czech immigration periods in Argentina.'},
  {'question': 'Can you tell me more about each period of Czech immigration?',
   'answer': 'The first one is slightly before to the World War 1, the second one from 1920 t

In [21]:
print(len(WikiDialog_LLM))
print(len(WikiDialog_LLM_instruction))
print(len(WikiDialog_LLM_instruction_passage_title))
print(len(WikiDialog_LLM_shot))

100
100
100


In [25]:
with open(f'WikiDialog_LLM_{num}.json', 'w') as json_file:
    json.dump(WikiDialog_LLM, json_file, indent=2)

In [26]:
with open(f'WikiDialog_LLM_instruction_{num}.json', 'w') as json_file:
    json.dump(WikiDialog_LLM_instruction, json_file, indent=2)

In [27]:
with open(f'WikiDialog_LLM_instruction_passage_title_{num}.json', 'w') as json_file:
    json.dump(WikiDialog_LLM_instruction_passage_title, json_file, indent=2)

In [44]:
with open(f'WikiDialog_LLM_shot_{num}.json', 'w') as json_file:
    json.dump(WikiDialog_LLM_shot, json_file, indent=2)